## Check the tables we need exist.

In [0]:
%sql
SELECT * FROM kenworkspace.tw_stocks_db.silver_daily

In [0]:
%sql
SELECT * FROM kenworkspace.tw_stocks_db.silver_mvrh_monthly

## Main part: final gold table for future ML.

In [0]:
%sql
CREATE OR REPLACE TABLE kenworkspace.tw_stocks_db.gold_ml_mvrh_daily AS
  --CTE 1
  WITH monthly_mvrh AS (
      SELECT
        date AS mvrh_month_date,
        low AS mvrh_low,
        high AS mvrh_high,
        from_mvrh_percent
      FROM kenworkspace.tw_stocks_db.silver_mvrh_monthly
      WHERE mvrh = true
  ),
  --CTE 2
  trade_days AS (
    SELECT
      date,
      ROW_NUMBER() OVER (ORDER BY date) AS trade_day_seq
    FROM kenworkspace.tw_stocks_db.silver_daily
  ),
  --CTE 3
  mvrh_month_last_trade AS (
    SELECT
      DATE_TRUNC('month', date) AS mvrh_month_date,
      MAX(date) AS last_trade_date_of_mvrh_month
    FROM kenworkspace.tw_stocks_db.silver_daily
    GROUP BY DATE_TRUNC('month', date)
  ),
  --CTE 4
  daily_with_prev AS (
    SELECT
      d.date,
      d.close,
      d.60fhr_percent,
      d.60flr_percent,
      mm.from_mvrh_percent,
      mm.mvrh_low,
      mm.mvrh_high,
      mm.mvrh_month_date AS last_mvrh_month_date,
      mmlt.last_trade_date_of_mvrh_month,
      td_d.trade_day_seq AS d_trade_seq,
      td_m.trade_day_seq AS mvrh_trade_seq,
      (td_d.trade_day_seq - td_m.trade_day_seq) AS days_from_mvrh
    FROM kenworkspace.tw_stocks_db.silver_daily d
    LEFT JOIN LATERAL (
      SELECT *
      FROM monthly_mvrh mm
      WHERE mm.mvrh_month_date < DATE_TRUNC('month', d.date)
      ORDER BY mm.mvrh_month_date DESC
      LIMIT 1
    ) mm
    LEFT JOIN mvrh_month_last_trade mmlt
      ON mmlt.mvrh_month_date = mm.mvrh_month_date
    LEFT JOIN trade_days td_d
      ON td_d.date = d.date
    LEFT JOIN trade_days td_m
      ON td_m.date = mmlt.last_trade_date_of_mvrh_month
  ),
  --CTE 5
  filtered_daily AS (
    SELECT *
    FROM daily_with_prev
    WHERE
      date IS NOT NULL
      AND close IS NOT NULL
      AND 60fhr_percent IS NOT NULL
      AND 60flr_percent IS NOT NULL
      AND from_mvrh_percent IS NOT NULL
      AND mvrh_low IS NOT NULL
      AND mvrh_high IS NOT NULL
      AND last_mvrh_month_date IS NOT NULL
  )

  SELECT
    date,
    from_mvrh_percent,
    days_from_mvrh,
    ROUND((close / mvrh_low - 1) * 100, 2) AS from_mvrh_low_percent,
    ROUND((close / mvrh_high - 1) * 100, 2) AS from_mvrh_high_percent,
    (60fhr_percent > 20 AND 60flr_percent > -10) AS 60_days_profitable
  FROM filtered_daily



In [0]:
%sql
use catalog `kenworkspace`; 

select * from `tw_stocks_db`.`gold_ml_mvrh_daily`
where `60_days_profitable` = true

In [0]:
%sql
use catalog `kenworkspace`; 

select * from `tw_stocks_db`.`gold_ml_mvrh_daily`